In [78]:
import names
import pickle
import crypto
#Setup 
#n Voters, m Candidates
n = 100
m = 4
# Generate random candidate names
P = np.array([names.get_full_name() for i in range(m)])
# Public key ck and private key sk generation given security parameter l
ck, sk = crypto.key_gen(100)
p,q,g,h = ck
# Commit and Decommit functions
Com = lambda ck, M, r: crypto.encrypt(M, ck, r)
Dec = lambda sk, C, r: crypto.decrypt(C, sk, r)
# Choose random permutations for each side of each ballot
perm = np.array([[np.random.permutation(m) for a in range(2)] for i in range(n)])
# Choose unique vote codes
C = np.array([[np.random.permutation(range(2*m*l + a*m + 1, 2*m*l + (a+1)*m + 1)) for a in range(2)] for l in range(1, n+1)])
x = np.max([2*m-1, n+1])
# Create array of powers of x. The (i+1)-th power indicates the (i+1)-th rank/position
R = []
temp = 1
for i in range(m):
    R.append(temp)
    temp *= x
R = np.array(R)
# Commit vote code values and apply permutation
t = np.array([[[random.randint(1,q-1) for j in range(m)] for a in range(2)] for i in range(n)])
U = np.array([[C[i][a][perm[i][a]] for a in range(2)] for i in range(n)])
U = np.array([[[Com(ck, U[i][a][j], t[i][a][j]) for j in range(m)] for a in range(2)] for i in range(n)])
# Generate double-ballots: s[i] contains the tuple (tag, s0, s1) of voter i
s = list([(i+1, [(P[k], C[i,0,k]) for k in range(m)], [(P[k], C[i,1,k]) for k in range(m)]) for i in range(n)])
# Commit ranks and apply same permutation as before
r = np.array([[[random.randint(1,q-1) for j in range(m)] for a in range(2)] for i in range(n)])
E = np.array([[R[perm[i,a]] for a in range(2)] for i in range(n)])
E = np.array([[np.array([Com(ck, E[i,a,j], r[i,a,j]) for j in range(m)]) for a in range(2)] for i in range(n)])
# Gather the public info and upload to BB
Publ = [(i+1, U[i], E[i]) for i in range(n)]
Pub = {
    "ck": ck,
    "P": P,
    "n": n, 
    "Publ": Publ
}
with open('BB', 'wb') as fbb:
    pickle.dump(Pub, fbb)
# # Saving ballots to 'ballots'
with open('ballots', 'wb') as f:
    pickle.dump(s, f)

In [79]:
### EA private info
info = {
    "ck":ck,
    "sk":sk,
    "s": s,
    "n": n,
    "m": m,
    "t": t,
    "r": r,
    "C": C,
    "perm": perm
}
with open('privateInfo', 'wb') as f:
    pickle.dump(info, f)